In [1]:
!pip install redis requests beautifulsoup4 sentence-transformers langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 806.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [2]:
pip install -U langchain-community redis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [3]:
import redis
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.redis import Redis
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Connect to Redis Cloud (Replace with your Redis Cloud credentials)
redis_host = "redis-18847.c16.us-east-1-2.ec2.redns.redis-cloud.com"
redis_port = 18847  # Replace with your port
redis_password = "ZhIcpw65p96DNC1wtxiftOuGwLI73Qpm"  # Replace with your password

redis_conn = redis.StrictRedis(
    host=redis_host,
    port=redis_port,
    password=redis_password,
    decode_responses=True
)
redis_url = f"redis://:{redis_password}@{redis_host}:{redis_port}"

# Step 2: Initialize the embedding model and vector store
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Redis(
    redis_url=redis_url,
    embedding=embedding_model,
    index_name="umiacs"  # Optional: Specify an index name
)


# URL of the Special:AllPages page on the wiki
all_pages_url = 'https://wiki.umiacs.umd.edu/umiacs/index.php/Special:AllPages'
response = requests.get(all_pages_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Create a set of unique pages on the wiki
pageset = set()
for link in soup.find_all('a'):
    href = link.get('href')
    # Ensure the link is valid and points to an internal page
    if href and href.startswith('/umiacs/index.php'):
        # Build the full URL for the wiki page
        full_url = 'https://wiki.umiacs.umd.edu' + href
        # Add the unique URL to the set, following redirects
        response = requests.get(full_url, allow_redirects=True)
        response_url = response.url
        pageset.add(response_url)

# List of URLs for HTML pages
html_links = list(pageset)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<ipython-input-3-092543cdb4d7>:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Rigid Chunking

Rigid chunkens where are we encode the text into tokens and limit the token size to 512.

Problem Faced: Sentences are cut mid-way in the database

In [4]:
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Ensure NLTK sentence tokenizer data is available
nltk.download("punkt")

# Step 2: Token-based chunking with overlap that respects sentence boundaries
MODEL_MAX_TOKENS = 128  # Max token limit for the MiniLM model
OVERLAP = 40  # Reduced overlap to fit within model constraints

def chunk_text_with_overlap(text, max_tokens=MODEL_MAX_TOKENS, overlap=OVERLAP):
    """Chunks text strictly within the model's token limits using encoding."""
    # Encode the entire text into token IDs
    input_ids = tokenizer.encode(text, add_special_tokens=False)
    chunks = []

    # Create chunks directly from the token IDs with overlap
    for i in range(0, len(input_ids), max_tokens - overlap):
        chunk_ids = input_ids[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text)

    return chunks

# Step 3: Function to fetch HTML content
def fetch_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return None


# Step 5: Process each page, generate embeddings, and store them in Redis
def process_and_store_page(url):
    html_content = fetch_html_from_url(url)
    if html_content:
        soup = BeautifulSoup(html_content, "html.parser")
        text_content = soup.get_text(separator=" ", strip=True)
        chunks = chunk_text_with_overlap(text_content)
        for chunk in chunks:
            embedding = embedding_model.embed_query(chunk)
            vector_store.add_texts([chunk], embeddings=[embedding])

# Step 6: Process all pages and store embeddings in Redis
for url in html_links:
    process_and_store_page(url)

print("All pages processed and embeddings stored in Redis!")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


Failed to fetch https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:Privacy_policy: 404 Client Error: Not Found for url: https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:Privacy_policy
Failed to fetch https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:General_disclaimer: 404 Client Error: Not Found for url: https://wiki.umiacs.umd.edu/umiacs/index.php/UMIACS:General_disclaimer
All pages processed and embeddings stored in Redis!


## Recursive Chunking
Split the text based on a series of text separators to ensure we do not have random chunks that are cut-halfway through the sentence.

In [ ]:
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nltk
from bs4 import BeautifulSoup
import requests

# Initialize the tokenizer and other components
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
MODEL_MAX_TOKENS = 128  # Max token limit for the MiniLM model
OVERLAP = 30  # Reduced overlap to fit within model constraints

# Function to calculate the max characters based on token limit
def calculate_max_characters(text, max_tokens=MODEL_MAX_TOKENS):
    """Estimate max characters for RecursiveCharacterTextSplitter based on token count."""
    tokens = tokenizer.tokenize(text)
    avg_char_per_token = len(text) / len(tokens)
    #return int(max_tokens * avg_char_per_token)
    return 3

# Step 2: Define the text splitter using RecursiveCharacterTextSplitter
max_chars = calculate_max_characters("sample text", max_tokens=MODEL_MAX_TOKENS)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=max_chars,
    chunk_overlap=OVERLAP,
    length_function=len
)

# Step 3: Function to fetch HTML content
def fetch_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return None

# Step 5: Process each page, generate embeddings, and store them in Redis
def process_and_store_page(url):
    html_content = fetch_html_from_url(url)
    if html_content:
        soup = BeautifulSoup(html_content, "html.parser")
        text_content = soup.get_text(separator=" ", strip=True)

        # Use the text splitter to chunk the text
        chunks = text_splitter.split_text(text_content)

        for chunk in chunks:
            # Verify token count is within model limit
            chunk_ids = tokenizer.encode(chunk, add_special_tokens=False)
            if len(chunk_ids) > MODEL_MAX_TOKENS:
                chunk_ids = chunk_ids[:MODEL_MAX_TOKENS]
                chunk = tokenizer.decode(chunk_ids, skip_special_tokens=True)

            # Embed and store the chunk
            embedding = embedding_model.embed_query(chunk)
            vector_store.add_texts([chunk], embeddings=[embedding])

# Step 6: Process all pages and store embeddings in Redis
for url in html_links:
    process_and_store_page(url)

print("All pages processed and embeddings stored in Redis!")


In [ ]:
# Generate query embedding
query = "sbatch jobs"
query_embedding = embedding_model.embed_query(query)  # Ensure this is a valid vector

# Step: Perform similarity search by vector
try:
    # Use the correct syntax for similarity search by vector
    results = vector_store.similarity_search_by_vector(embedding=query_embedding, k=5)

    # Correctly extract text from Document objects
    for doc in results:
        print(doc.page_content)  # Access the content of the Document object
except Exception as e:
    print(f"Error during similarity search: {e}")

: ss # sbatch - - qos = default # set qos, this will determine what resources can be requested # sbatch - - nodes = 2 # number of nodes to allocate for your job # sbatch - - ntasks = 4 # request 4 cpu cores be reserved for your node total # sbatch - - ntasks - per - node = 2 # request 2 cpu cores be reserved per node # sbatch - - mem = 1g # memory required by job ; if unit is not specified mb will be assumed. for multi - node jobs, this argument allocates this much memory * per node * srun - - nodes = 1 - - mem = 512m bash - c " hostname ; python3 - - version " & # use srun to invoke commands within your job ; using an ' & ' srun - - nodes = 1 - - mem = 512m bash - c " hostname ; python3 - - version " & # will background the process allowing them to run concurrently wait # wait for any background processes to complete # once the end of the batch script is reached your job allocation will be revoked another useful thing to know is that you can pass additional arguments into your sbatch 

### In case we want to delete the vectors

In [4]:
def delete_all_vectors(redis_conn, index_name):
    """Deletes all keys in the given Redis namespace."""
    try:
        # Use SCAN to retrieve all keys that belong to the index namespace
        keys = redis_conn.scan_iter(f"{index_name}:*")
        for key in keys:
            redis_conn.delete(key)  # Delete each key
        print(f"All vectors from index '{index_name}' have been deleted.")
    except Exception as e:
        print(f"Error deleting vectors: {e}")

# Example usage
delete_all_vectors(redis_conn, "umiacs")

All vectors from index 'umiacs' have been deleted.
